In [913]:
import sqlite3
import numpy as np
import pandas as pd

con = sqlite3.connect("database.sqlite")

In [914]:
# country_id="21518" and league_id="21518" define La Liga
# id="43040" or team_api_id="8633" define Real Madrid
# id="43039" or team_api_id="9783" define RC Deportivo de La Coruña (opponent of Real Madrid in stage 38)
df = pd.read_sql_query('SELECT id, home_team_goal, away_team_goal, '
                       'home_team_api_id, away_team_api_id, '
                       'B365H, B365D, B365A '
                       'FROM Match '
                       'WHERE country_id!="21518" '
                       'ORDER BY season, stage ASC', con)
df_test = pd.read_sql_query('SELECT id, home_team_goal, away_team_goal, '
                       'home_team_api_id, away_team_api_id, '
                       'B365H, B365D, B365A '
                       'FROM Match '
                       'WHERE country_id="21518" '
                       'ORDER BY season, stage ASC', con)
df.dropna(inplace=True)
df_test.dropna(inplace=True)

In [915]:
def prepareDF(dataframe):
    teams = dataframe['home_team_api_id'].unique()
    points = {}
    bets = {}
    points_against_teams = {}
    bets_against_teams = {}
    for team_id in teams:
        points[team_id] = {}
        bets[team_id] = {}
        points_against_teams[team_id] = {}
        bets_against_teams[team_id] = {}
        for team_id_inner in teams:
                if team_id != team_id_inner:
                    points_against_teams[team_id][team_id_inner] = {}
                    bets_against_teams[team_id][team_id_inner] = {}
    for i, value in enumerate(dataframe.values):
        home_team_api_id = dataframe['home_team_api_id'].values[i]
        away_team_api_id = dataframe['away_team_api_id'].values[i]
        home_team_goal = dataframe['home_team_goal'].values[i]
        away_team_goal = dataframe['away_team_goal'].values[i]
        id_of_match = dataframe['id'].values[i]
        bets[home_team_api_id][id_of_match] = dataframe['B365H'].values[i]
        bets[away_team_api_id][id_of_match] = dataframe['B365A'].values[i]
        bets_against_teams[home_team_api_id][away_team_api_id][id_of_match] = dataframe['B365H'].values[i]
        bets_against_teams[away_team_api_id][home_team_api_id][id_of_match] = dataframe['B365A'].values[i]
        if home_team_goal > away_team_goal:
            points_against_teams[home_team_api_id][away_team_api_id][id_of_match] = 3
            points_against_teams[away_team_api_id][home_team_api_id][id_of_match] = 0
            points[home_team_api_id][id_of_match] = 3
            points[away_team_api_id][id_of_match] = 0
        elif home_team_goal < away_team_goal:
            points_against_teams[home_team_api_id][away_team_api_id][id_of_match] = 0
            points_against_teams[away_team_api_id][home_team_api_id][id_of_match] = 3
            points[home_team_api_id][id_of_match] = 0
            points[away_team_api_id][id_of_match] = 3
        else:
            points_against_teams[home_team_api_id][away_team_api_id][id_of_match] = 1
            points_against_teams[away_team_api_id][home_team_api_id][id_of_match] = 1
            points[home_team_api_id][id_of_match] = 1
            points[away_team_api_id][id_of_match] = 1
    bet_ratio = []
    home_pitlf = []
    away_pitlf = []
    home_vs_away_pitlf = []
    away_vs_home_pitlf = []
    home_points_avg = []
    away_points_avg = []
    home_points_avg_vs_away = []
    away_points_avg_vs_home = []
    home_bets_avg = []
    away_bets_avg = []
    home_vs_away_bets_avg = []
    away_vs_home_bets_avg = []
    for i, value in enumerate(dataframe.values):
        home_team_api_id = dataframe['home_team_api_id'].values[i]
        away_team_api_id = dataframe['away_team_api_id'].values[i]
        id_of_match = dataframe['id'].values[i]
        bet_ratio = dataframe['B365H'].values[i] / dataframe['B365A'].values[i]

        pitlf, avgpoints = stats_general(id_of_match,points[home_team_api_id])
        home_pitlf.append(pitlf)
        home_points_avg.append(avgpoints)
        pitlf, avgpoints = stats_general(id_of_match,points[away_team_api_id])
        away_pitlf.append(pitlf)
        away_points_avg.append(avgpoints)
        pitlf, avgpoints = stats_general(id_of_match,points_against_teams[home_team_api_id][away_team_api_id])
        home_vs_away_pitlf.append(pitlf)
        home_points_avg_vs_away.append(avgpoints)
        pitlf, avgpoints = stats_general(id_of_match,points_against_teams[away_team_api_id][home_team_api_id])
        away_vs_home_pitlf.append(pitlf)
        away_points_avg_vs_home.append(avgpoints)

        home_bets_avg.append(stats_bets(id_of_match, bets[home_team_api_id]))
        away_bets_avg.append(stats_bets(id_of_match, bets[away_team_api_id]))
        home_vs_away_bets_avg.append(stats_bets(id_of_match, bets_against_teams[home_team_api_id][away_team_api_id]))
        away_vs_home_bets_avg.append(stats_bets(id_of_match, bets_against_teams[away_team_api_id][home_team_api_id]))
    dataframe['bet_ratio'] = bet_ratio
    dataframe['home_points_avg'] = home_points_avg
    dataframe['away_points_avg'] = away_points_avg
    dataframe['home_points_avg_vs_away'] = home_points_avg_vs_away
    dataframe['away_points_avg_vs_home'] = away_points_avg_vs_home
    dataframe['home_pitlf'] = home_pitlf
    dataframe['away_pitlf'] = away_pitlf
    dataframe['home_vs_away_pitlf'] = home_vs_away_pitlf
    dataframe['away_vs_home_pitlf'] = away_vs_home_pitlf
    dataframe['home_bets_avg'] = home_bets_avg
    dataframe['away_bets_avg'] = away_bets_avg
    dataframe['home_vs_away_bets_avg'] = home_vs_away_bets_avg
    dataframe['away_vs_home_bets_avg'] = away_vs_home_bets_avg
    dataframe.dropna(inplace=True)

def stats_general(id_of_match, points):
    pitlf = 0
    avgpoints = 0
    for j, key in enumerate(points.keys()):
        if key == id_of_match:
            if j-5 >= 0:
                for k, point in enumerate(points.values()):
                    if k < j:
                        avgpoints += point
                        if j - 5 < k:
                            pitlf += point
                avgpoints = (avgpoints / (j +1))
                return pitlf, avgpoints
            else:
                return np.nan, np.nan

def stats_bets(id_of_match, bets):
    avgbets = 0
    for j, key in enumerate(bets.keys()):
        if key == id_of_match:
            if j-5 >= 0:
                for k, bet in enumerate(bets.values()):
                    if k < j:
                        if j - 5 < k:
                            avgbets += bet
                avgbets = (avgbets / 5)
                return avgbets
            else:
                return np.nan

In [916]:
prepareDF(df)
prepareDF(df_test)

In [917]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch
import torch.nn as nn
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO, Predictive

In [918]:
class BayesianRegression(PyroModule):
    def __init__(self, input_features=8, h1=20, h2=20):
        super().__init__()
        self.input_features=input_features
        self.fc1 = PyroModule[nn.Linear](input_features, h1)
        self.fc1.weight = PyroSample(dist.Normal(0., 1.).expand([h1, input_features]).to_event(2))
        self.fc1.bias = PyroSample(dist.Normal(0., 1.).expand([h1]).to_event(1))
        self.fc2 = PyroModule[nn.Linear](h1, h2)
        self.fc2.weight = PyroSample(dist.Normal(0., 1.).expand([h2, h1]).to_event(2))
        self.fc2.bias = PyroSample(dist.Normal(0., 1.).expand([h2]).to_event(1))
        self.fc3 = PyroModule[nn.Linear](h2, 1)
        self.fc3.weight = PyroSample(dist.Normal(0., 1.).expand([1, h2]).to_event(2))
        self.fc3.bias = PyroSample(dist.Normal(0., 1.).expand([1]).to_event(1))
        self.relu = nn.ReLU()

    def forward(self, x, y=None):
        x = x.reshape(-1, self.input_features)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        mu = self.fc3(x).squeeze()
        sigma = pyro.sample("sigma", dist.Uniform(0., 1.))
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma), obs=y)
        return mu

class BayesianRegression2(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features=in_features
        self.linear = PyroModule[nn.Linear](in_features, out_features)
        self.linear.weight = PyroSample(dist.Normal(1., 2.).expand([out_features, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0., 1.).expand([out_features]).to_event(1))

    def forward(self, x, y=None):
        x = x.reshape(-1, self.in_features)
        sigma = pyro.sample("sigma", dist.Uniform(0., 1.))
        mean = self.linear(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        return mean

In [919]:
used_parameters = ['home_vs_away_bets_avg']

x_data_raw = []
x_data_test_raw = []
for param in used_parameters:
    x_data_raw.append(df[param].values)
    x_data_test_raw.append(df_test[param].values)

In [920]:
x_data = torch.tensor(x_data_raw, dtype=torch.float)
x_data_test = torch.tensor(x_data_test_raw, dtype=torch.float)
y_data = torch.tensor(df['B365H'].values, dtype=torch.float)


In [921]:
model = BayesianRegression2(in_features=len(used_parameters), out_features=1)
guide = AutoDiagonalNormal(model)
adam = pyro.optim.Adam({"lr": 1e-3})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

In [922]:
num_iterations = 10000
pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(x_data, y_data)
    if j % 500 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_data)))
    if 0 < loss < 100:
        break

[iteration 0001] loss: 57461.2345
[iteration 0501] loss: 13655.6238
[iteration 1001] loss: 12660.2968
[iteration 1501] loss: 12630.1898
[iteration 2001] loss: 12685.1490
[iteration 2501] loss: 12226.0016
[iteration 3001] loss: 11863.9638
[iteration 3501] loss: 11800.0287
[iteration 4001] loss: 11535.5892
[iteration 4501] loss: 11503.7481
[iteration 5001] loss: 11481.8400
[iteration 5501] loss: 11456.8936
[iteration 6001] loss: 11452.0748
[iteration 6501] loss: 11450.6734
[iteration 7001] loss: 11516.9750
[iteration 7501] loss: 11470.4673
[iteration 8001] loss: 11443.0739
[iteration 8501] loss: 11447.5261
[iteration 9001] loss: 11446.2112
[iteration 9501] loss: 11449.3866


In [923]:
guide.requires_grad_(False)

predictive = Predictive(model, guide=guide, num_samples=1000, return_sites=("linear.weight", "obs", "_RETURN"))
preds = predictive(x_data_test)

y_pred = preds['obs'].T.detach().numpy().mean(axis=1)
y_std = preds['obs'].T.detach().numpy().std(axis=1)

In [924]:
weight = preds["linear.weight"]
weight = weight.reshape(weight.shape[0], len(used_parameters))
for i, param in enumerate(used_parameters):
    print(param + ': ' + str(np.round(torch.mean(weight[:, i]).item(), 4)))

home_vs_away_bets_avg: 0.7631


In [925]:
error = 0
within = 0
for i, bet in enumerate(y_pred):
    error += abs(bet - df_test['B365H'].values[i])
    if abs(bet - df_test['B365H'].values[i]) < 0.5:
        within += 1
avg_error = error / len(y_pred)
within_percent = within / len(y_pred)
print(avg_error)
print(within_percent)


0.6631990832297175
0.6900539707016191
